In [0]:
USE mgiglia.hls_genie;

In [0]:
"MARITAL_M",
      "MARITAL_S",
      "RACE_asian",
      "RACE_black",
      "RACE_hawaiian",
      "RACE_other",
      "RACE_white",
      "ETHNICITY_hispanic",
      "ETHNICITY_nonhispanic",
      "GENDER_F",
      "GENDER_M",
      "INCOME",
      "BASE_ENCOUNTER_COST",
      "TOTAL_CLAIM_COST",
      "PAYER_COVERAGE",
      "enc_length",
      "ENCOUNTERCLASS_ambulatory",
      "ENCOUNTERCLASS_emergency",
      "ENCOUNTERCLASS_hospice",
      "ENCOUNTERCLASS_inpatient",
      "ENCOUNTERCLASS_outpatient",
      "ENCOUNTERCLASS_wellness",
      "age_at_encounter"

In [0]:
SHOW TABLES;

In [0]:
SELECT DISTINCT
  p.patient_id
  ,p.birth_date
  ,p.gender
  ,p.ethnicity
  ,p.marital
  ,p.race
  ,p.income
  ,e.start as enc_start
  ,e.stop as enc_stop
  ,e.total_claim_cost
  ,e.payer_coverage
  ,e.encounter_class
FROM 
  patients as p
  ,encounters as e JOIN payers ON e.patient_id = p.patient_id
;

In [0]:
CREATE OR REPLACE FUNCTION age_at_encounter(
  birth_date DATE COMMENT 'Patient birth date'
  ,enc_date DATE COMMENT 'Encounter date or encounter start date'
) 
RETURNS INT COMMENT 'Age at encounter'
RETURN 
  CASE 
    WHEN MONTH(enc_date) > MONTH(birth_date) OR (MONTH(enc_date) = MONTH(birth_date) AND DAY(enc_date) >= DAY(birth_date)) 
    THEN YEAR(enc_date) - YEAR(birth_date) 
    ELSE YEAR(enc_date) - YEAR(birth_date) - 1 
  END;

In [0]:
select 
  p.patient_id
  ,p.birth_date
  ,e.start
  ,age_at_encounter(p.birth_date, e.start) as age_at_encounter 
from 
  patients as p join 
  encounters as e on p.patient_id = e.patient_id
;

In [0]:
DROP FUNCTION race_one_hot;

In [0]:
CREATE OR REPLACE FUNCTION race_one_hot(
  race STRING COMMENT 'Patient race'
)
RETURNS STRUCT<
  RACE_asian INTEGER,
  RACE_black INTEGER,
  RACE_hawaiian INTEGER,
  RACE_other INTEGER,
  RACE_white INTEGER
> COMMENT 'One hot encoded race'
RETURN STRUCT(
  CASE WHEN race = 'asian' THEN 1 ELSE 0 END AS RACE_asian,
  CASE WHEN race = 'black' THEN 1 ELSE 0 END AS RACE_black,
  CASE WHEN race = 'hawaiian' THEN 1 ELSE 0 END AS RACE_hawaiian,
  CASE WHEN race = 'other' THEN 1 ELSE 0 END AS RACE_other,
  CASE WHEN race = 'white' THEN 1 ELSE 0 END AS RACE_white
)
;

In [0]:
SELECT
  p.patient_id
  ,p.race
  ,race_one_hot(p.race) as race_one_hot
FROM patients p

In [0]:
CREATE OR REPLACE FUNCTION marital_one_hot(
  marital STRING COMMENT 'Patient marital status'
)
RETURNS INTEGER COMMENT 'One hot encoded martial status'
RETURN CASE WHEN marital = 'S' THEN 1 ELSE 0 END
;

In [0]:
SELECT 
  marital
  ,marital_one_hot(marital) as marital_one_hot
FROM patients;

In [0]:
CREATE OR REPLACE FUNCTION ethnicity_one_hot(ethnicity STRING)
RETURNS STRUCT<ETHNICITY_hispanic: INT, ETHNICITY_nonhispanic: INT>
RETURN 
  CASE 
    WHEN ethnicity = 'hispanic' THEN named_struct('ETHNICITY_hispanic', 1, 'ETHNICITY_nonhispanic', 0)
    ELSE named_struct('ETHNICITY_hispanic', 0, 'ETHNICITY_nonhispanic', 1)
  END;

In [0]:
select
  ethnicity
  ,ethnicity_one_hot(ethnicity) as ethnicity_one_hot
from patients

In [0]:
create or replace function gender_one_hot(gender string)
returns struct<GENDER_F: int, GENDER_M: int>
return named_struct(
  'GENDER_F', case when gender = 'F' then 1 else 0 end,
  'GENDER_M', case when gender = 'M' then 1 else 0 end
)

In [0]:
SELECT 
  gender
  ,gender_one_hot(gender)
FROM patients;

In [0]:
CREATE OR REPLACE FUNCTION encounterclass_one_hot(encounterclass STRING)
RETURNS STRUCT<
  ENCOUNTERCLASS_ambulatory INT,
  ENCOUNTERCLASS_emergency INT,
  ENCOUNTERCLASS_hospice INT,
  ENCOUNTERCLASS_inpatient INT,
  ENCOUNTERCLASS_outpatient INT,
  ENCOUNTERCLASS_wellness INT
>
RETURN (
  CASE encounterclass
    WHEN 'ambulatory' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 1, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'emergency' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 1, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'hospice' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 1, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'inpatient' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 1, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'outpatient' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 1, 'ENCOUNTERCLASS_wellness', 0)
    WHEN 'wellness' THEN NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 1)
    ELSE NAMED_STRUCT('ENCOUNTERCLASS_ambulatory', 0, 'ENCOUNTERCLASS_emergency', 0, 'ENCOUNTERCLASS_hospice', 0, 'ENCOUNTERCLASS_inpatient', 0, 'ENCOUNTERCLASS_outpatient', 0, 'ENCOUNTERCLASS_wellness', 0)
  END
);

In [0]:
select 
  encounter_class
  ,encounterclass_one_hot(encounter_class) as encounterclass_one_hot
from encounters;